<a href="https://colab.research.google.com/github/marziyeahmadi/ofd/blob/main/OFD_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
!pip install pulp
import numpy as np
import pandas as pd
import pulp



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 41.6 MB/s eta 0:00:00


In [ ]:
# Simulation Parameters (Table 1)
products = [
    {"id": 1, "price": 10.00, "station": 1, "dine_mean": 100, "dine_std": 20, "ofd_mean": 100, "ofd_std": 20},
    {"id": 2, "price": 11.00, "station": 1, "dine_mean": 80, "dine_std": 16, "ofd_mean": 80, "ofd_std": 16},
    {"id": 3, "price": 12.00, "station": 2, "dine_mean": 100, "dine_std": 20, "ofd_mean": 100, "ofd_std": 20},
    {"id": 4, "price": 13.00, "station": 2, "dine_mean": 80, "dine_std": 16, "ofd_mean": 80, "ofd_std": 16}
]

price_increase = [0, 0.15, 0.30, 0.45]  # No change, 15%, 30%, 45%
delivery_commission = 0.30


In [ ]:
# Capacity scenarios
capacity_scenarios = [(200 - i*10, 400 - i*20) for i in range(16)]

In [ ]:
# Use PULP_CBC_CMD solver with `msg=False` to avoid subprocess calls
solver = pulp.PULP_CBC_CMD(msg=False)

In [ ]:
# Forecasted demand (assuming this is incorporated as the mean)
forecast_dine = [p["dine_mean"] for p in products]
forecast_ofd = [p["ofd_mean"] for p in products]

In [ ]:
# Run simulation
results_no_rm = []
results_rm = []

In [ ]:
for product_cap, station_cap in capacity_scenarios:
    dine_in_qty, dine_in_revenue, ofd_qty, ofd_revenue = 0, 0, 0, 0
    dine_in_qty_rm, dine_in_revenue_rm, ofd_qty_rm, ofd_revenue_rm = 0, 0, 0, 0

    for _ in range(100):
        # Randomize actual demand around forecast
        demand_dine = [max(0, np.random.normal(forecast_dine[i], products[i]["dine_std"])) for i in range(4)]
        demand_ofd = [max(0, np.random.normal(forecast_ofd[i], products[i]["ofd_std"])) for i in range(4)]

        # No RM scenario - adjust dine-in if OFD exceeds capacity
        total_demand = np.array(demand_dine) + np.array(demand_ofd)
        excess_ofd = np.maximum(0, total_demand - product_cap)
        dine_in_fulfilled = np.maximum(0, demand_dine - excess_ofd)

        dine_in_qty += sum(dine_in_fulfilled)
        dine_in_revenue += sum(np.array(dine_in_fulfilled) * np.array([p["price"] for p in products]))
        ofd_qty += sum(demand_ofd)
        ofd_revenue += sum(np.array(demand_ofd) * np.array([p["price"] for p in products]))

        # RM Scenario using integer programming
        model = pulp.LpProblem("RM_Pricing", pulp.LpMaximize)
        x = pulp.LpVariable.dicts("x", [(i, k) for i in range(4) for k in range(4)], 0, cat='Integer')

        # Objective: Maximize revenue
        model += pulp.lpSum([(products[i]["price"]*(1 + price_increase[k])) * x[(i,k)] for i in range(4) for k in range(4)])

        # Demand constraints
        for i in range(4):
            model += pulp.lpSum([x[(i,k)] for k in range(4)]) <= demand_ofd[i]

        # Capacity constraints per product
        for i in range(4):
            model += pulp.lpSum([x[(i,k)] for k in range(4)]) <= max(0, product_cap - demand_dine[i])

        # Capacity constraints per station
        for s in [1,2]:
            model += pulp.lpSum([x[(i,k)] for i in range(4) if products[i]["station"] == s for k in range(4)]) <= max(0, station_cap - sum(demand_dine[i] for i in range(4) if products[i]["station"] == s))

        # One price per product
        for i in range(4):
            model += pulp.lpSum([x[(i,k)] for k in range(4)]) <= demand_ofd[i]

        model.solve(solver)

        # RM results with error handling
        ofd_qty_rm_run = sum(x[(i,k)].varValue or 0 for i in range(4) for k in range(4))
        ofd_revenue_rm_run = sum((x[(i,k)].varValue or 0) * (products[i]["price"]*(1 + price_increase[k])) for i in range(4) for k in range(4))
        dine_in_qty_rm += sum(demand_dine)
        dine_in_revenue_rm += sum(np.array(demand_dine) * np.array([p["price"] for p in products]))
        ofd_qty_rm += ofd_qty_rm_run
        ofd_revenue_rm += ofd_revenue_rm_run

    # Average results
    results_no_rm.append([product_cap*4, dine_in_qty/100, dine_in_revenue/100, ofd_qty/100, ofd_revenue/100])
    results_rm.append([product_cap*4, dine_in_qty_rm/100, dine_in_revenue_rm/100, ofd_qty_rm/100, ofd_revenue_rm/100])


In [ ]:
# Convert results to DataFrame
results_no_rm_df = pd.DataFrame(results_no_rm, columns=['Total Capacity','Dine-in Qty','Dine-in Rev','OFD Qty','OFD Rev'])
results_rm_df = pd.DataFrame(results_rm, columns=['Total Capacity','Dine-in Qty','Dine-in Rev','OFD Qty','OFD Rev'])


In [ ]:
# Adjust for commission
results_no_rm_df['Net Rev'] = results_no_rm_df['Dine-in Rev'] + results_no_rm_df['OFD Rev']*(1 - delivery_commission)
results_rm_df['Net Rev'] = results_rm_df['Dine-in Rev'] + results_rm_df['OFD Rev']*(1 - delivery_commission)

In [ ]:
# Output results
print("Results without RM:")
print(results_no_rm_df)

print("\nResults with RM:")
print(results_rm_df)


Results without RM:
    Total Capacity  Dine-in Qty  Dine-in Rev     OFD Qty      OFD Rev  \
0              800   335.500512  3846.145697  363.144192  4158.999780   
1              760   318.509711  3657.527711  362.968760  4153.558406   
2              720   310.461426  3577.765646  361.333430  4131.722220   
3              680   287.178449  3306.862268  361.574363  4141.617464   
4              640   256.301664  2960.192138  365.791283  4187.190679   
5              600   228.362780  2636.932277  361.432963  4143.061412   
6              560   198.625223  2301.505526  356.157458  4077.610857   
7              520   162.056281  1873.952854  357.242096  4096.170264   
8              480   120.886701  1411.226360  363.450829  4154.402579   
9              440    88.036739  1031.243907  361.600874  4135.069588   
10             400    58.817970   688.623521  360.150332  4122.573867   
11             360    33.128929   389.558534  360.398134  4124.206565   
12             320    16.119715